<h1> Minimizando o risco de uma Carteira de Investimentos com otimização linear </h1>

<img src="https://media-exp1.licdn.com/dms/image/C4D12AQH4b_B99E8plQ/article-cover_image-shrink_720_1280/0/1559514788836?e=1629936000&v=beta&t=UOSOnwThfNUoyLrcYNW__SmXREmruEIGWh6pEY0WiuY">

<p>As decisões financeiras na prática não são tomadas em ambiente de total certeza com relação a seus resultados. Por essas decisões estarem fundamentalmente voltadas para o futuro, é imprescindível que se introduza a variável incerteza (risco) como um dos mais significativos aspectos do estudo das finanças corporativas. Saber mensurar corretamente o risco e retorno de um negócio, ou seja, saber exatamente com o que está lidando, é uma das principais habilidades que um empreendedor ou um gestor de fundos precisa ter para obter sucesso.</p>

<p> Como então mensuramos o risco? Primeiro precisamos definir uma medida de risco. De maneira geral, medimos o risco pelo quanto de capital precisamos adicionar à nossa posição de risco para termos uma posição aceitável. Por exemplo, suponha que você tenha 1000 reais para investir no mercado de ações e quer ganhar 1 milhão de reais em três anos. O grau de incerteza de obter esse retorno seria ridiculamente alto, (mas teoricamente seria possível, bastando investir no início do dia, na ação que vai dar maior retorno no final dia, e depois trocando para a ação que vai dar maior retorno no final do dia seguinte, sucessivamente) e deveríamos adicionar um capital bastante expressivo nesse investimento para obtermos uma posição aceitável de risco.</p>

<p> Existem várias maneiras de medir um risco de uma carteira de investimentos. Vamos avaliar neste artigo as três mais importantes: Modelo de Markowitz, Valor em Risco (VaR) e Valor em Risco Condicional (CVaR). Mas antes disso, vamos entender o que é ter uma carteira "coerente". A noção de coerência foi introduzida por Artzner et al e atualmente, é um conceito fundamental relacionado à aceitabilidade de uma medida de risco. A literatura introduz um número de propriedades que são usadas para determinar um medida de risco. As propriedades mais importantes para a medida de risco são: </p>

<ol>
    <li><b>Invariância à translação </b>: Se adicionarmos ou subtrairmos uma quantidade certa de nossa carteira, a medida de risco aumenta ou diminui. Matematicamente, se $A$ é o quanto queremos adicionar aos ganhos da carteira, $X$ e $p$ a nossa medida de risco, então $p(X+A)=p(X)+A$. Veja que, se trocarmos instrumentos de renda varável e alocarmos em renda fixa (nessa caso, $A$ é negativo) a carteira diminui o risco no mesmo montante.</li>
    <li><b>Subaditividade</b>: Na minha opinião, é a propriedade mais importante ao se avaliar uma medida de risco. Esta medida é intimamente relacionada com o efeito da diversificação do portfólio. A medida do risco total da carteira (conjunto de ativos) é menor ou igual que a medida do risco da soma individual dos ativos da carteira ($p(X_{1})+p(X_{2})<p(X_{1}+X_{2})$). É o princípio da Teoria Moderno do Portfólio, do grande Markowitz. E para quem gosta de avaliar o desempenho de uma carteira usando o VaR, desculpe decepcioná-los, mais <b>o VaR falha na subaditivade</b>. Isso significa que, minimizar o VaR não garante que você vá diversificar os investimentos da carteira considerada.</li>
    <li><b>Monotonicidade</b>:  Se os ganhos na carteira $X$ são menores que os da carteira $Y$ para todos os cenários possíveis, então o risco na carteira $X$ é menor que na carteira $Y$ (Se $X_{1}<X_{2}$, então $p(X_{1})<p(X_{2})$). Claro, num portfólio devidamente otimizado, se você quiser arriscar mais, espera que sua possibilidade de ganho seja maior. Se você é fã da Teoria de Markowitz, não vai ser fã dessa propriedade, pois infelizmente Markowitz falha na monotonicidade :(.</li>
    <li><b>Homogeneidade Positiva</b>: Ao aumentar o tamanho de cada posição da carteira o risco da carteira aumenta em igual proporção ($p(bX)=bp(X)$), sendo $b$ uma constante). Isso significa que, se você trocar a moeda de uma carteira, ou dobrar seu investimento em cada ação, seu risco aumentará na mesma proporção. Se você aposta 100 reais num jogo de poker e depois triplica a aposta, seu risco triplica também, pois você pode perder (ou ganhar) três vezes mais do que antes.</li>
</ol>

<h3> Importando bibliotecas </h3>

Para começar nosso projeto, vamos importar as bibliotecas necessárias.

In [1]:
import numpy as np #manipulação de matrizes/ álgebra linear
import pandas as pd #manipulação de dataframes
from datetime import datetime,timedelta #datas
import numpy.matlib
import matplotlib.pyplot as plt #imagens
import plotly.express as px #imagens interativas
import plotly.graph_objects as go
import pandas_datareader.data as web #importar dados históricos de ativos
import scipy.io #estatística
from scipy.optimize import linprog #algoritmo para otimização linear
from time import sleep 
from tqdm.notebook import tqdm #avaliar passar em um loop

In [2]:
import plotly
plotly.__version__

'5.9.0'

In [26]:
import utils 
start_time = time.time()
start = datetime.now() - timedelta(days=400)
end = datetime.now()

path_df = "s3://bbs-datalake/SourceZone/df_stocks_info.parquet"
df_stocks_info = pd.read_parquet(path_df)

stock_type = ['BDR', 'ON', 'PN', 'PNA', 'PNB', 'Subscrição', 'PNC', 'PND']
stock_sector = list(df_stocks_info["Setor"].unique())
stock_technical = ["Compra Forte", "Compra", "Venda", "Venda Forte", "Neutro"]

df_filter = df_stocks_info.query(
    "Tipo in @stock_type and Setor in @stock_sector and Mensal in @stock_technical"
)
stocks_codes = [i + ".SA" for i in df_filter.Códigos.unique()]
utils.collect_historico_stocks(stocks_codes, start, end)
end_time = time.time()
elapsed_time = end_time - start_time

[*********************100%%**********************]  981 of 981 completed


9 Failed downloads:
['1BHIA3.SA', '1WIZC3.SA', 'BPAN12.SA', '1ALOS3.SA', 'GOLL12.SA', '1HYPE3.SA']: Exception('%ticker%: No timezone found, symbol may be delisted')
['SEER3T.SA', 'CVCB3T.SA', 'ANIM3T.SA']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2022-11-26 06:03:23.368304 -> 2023-12-31 06:03:23.368304)')


In [27]:
elapsed_time

38.71325755119324

In [2]:
pd.set_option('display.max_columns', 500)

<h3> Coletando o histórico de ações </h3>

Selecionaremos as principais ações da bolsa de valores do Brasil.

In [3]:
from pandas_datareader import data as pdr
import yfinance as yfin


yfin.pdr_override()

In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time

In [6]:
url = 'https://br.investing.com/stock-screener/?sp=country::32|sector::a|industry::a|equityType::a|exchange::a%3Ceq_market_cap;1'
driver = webdriver.Chrome()
driver.get(url)
time.sleep(5)
list_df = []
c = True
i=0
while c:
    try:
        table_element = driver.find_element(By.XPATH, '//*[@id="resultsTable"]')
        table_html = table_element.get_attribute('outerHTML')
        list_df.append(pd.read_html(table_html, index_col=0, thousands='.', decimal=',', na_values=['-'])[0].reset_index(drop=True))
        driver.find_element(By.XPATH, '//*[@id="paginationWrap"]/div[3]/a').click()
        time.sleep(5)
        i+=1
        print('página', i)
    except Exception as e:
        # Trate o erro aqui, se necessário
        print(f"Número de páginas alcançadas: {e}")
        # Encerre o loop após o erro
        c = False
        
df = pd.concat(list_df)
df = df.iloc[:,:-1]

página 1
página 2
página 3
página 4
página 5
página 6
página 7
página 8
página 9
página 10
página 11
página 12
página 13
página 14
página 15
página 16
página 17
página 18
página 19
página 20
página 21
página 22
página 23
página 24
página 25
página 26
página 27
página 28
Número de páginas alcançadas: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="paginationWrap"]/div[3]/a"}
  (Session info: chrome=120.0.6099.130); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7D7F12142+3514994]
	(No symbol) [0x00007FF7D7B30CE2]
	(No symbol) [0x00007FF7D79D76AA]
	(No symbol) [0x00007FF7D7A21860]
	(No symbol) [0x00007FF7D7A2197C]
	(No symbol) [0x00007FF7D7A64EE7]
	(No symbol) [0x00007FF7D7A4602F]
	(No symbol) [0x00007FF7D7A628F6]
	(No symbol) [0x00007FF7D7A45D93]
	(No symbol) [0x00007FF7D7A14BDC]
	(No symbol) [0x00007FF7D7A1

In [5]:
col_str = ['Nome', 'Códigos', 'Bolsa', 'Setor', 'Indústria', '15 minutos', 'Hora', 'Diário', 'Semanal', 'Mensal']
for coluna in df.columns:
    if coluna not in col_str:
        if df[coluna].dtype == 'O':  # Verifica se a coluna é do tipo objeto (string)
            df[coluna] = pd.to_numeric(df[coluna], errors='coerce')
Tipos_depara = {'34':'BDR', '11':'FII', '3':'ON', '4':'PN', '5':'PNA', '6':'PNB', '31': 'BDR', 
                '12':'Subscrição', '33':'BDR', '32':'BDR', '35':'BDR', '7': 'PNC', '8': 'PND'} 
df['Tipo'] = df['Códigos'].str[4:].replace(to_replace='[^0-9]', value='', regex=True).replace(Tipos_depara)
df.to_parquet("df_stocks_info.parquet")

NameError: name 'df' is not defined

In [21]:
#stocks = [i+'.SA' for i in df_ON.Códigos.unique()]
df_stocks_infob = pd.read_parquet("s3://bbs-datalake/SourceZone/df_stocks_info.parquet")
start = datetime(2020,1,1)
end = datetime(2023,12,10)

stock_type = ['BDR', 'ON', 'PN', 'PNA', 'PNB', 'Subscrição', 'PNC', 'PND']
stock_sector = list(df_stocks_info["Setor"].unique())
stock_technical = ["Compra Forte", "Compra", "Venda", "Venda Forte", "Neutro"]

df_filter = df_stocks_info.query(
    "Tipo in @stock_type and Setor in @stock_sector and Mensal in @stock_technical"
)
stocks_codes = [i + ".SA" for i in df_filter.Códigos.unique()]

In [22]:
historico_stocks = web.DataReader(stocks_codes, start, end)['Adj Close']

[*********************100%%**********************]  981 of 981 completed


10 Failed downloads:
['1BHIA3.SA', '1WIZC3.SA', 'BPAN12.SA', '1ALOS3.SA', 'GOLL12.SA', '1HYPE3.SA']: Exception('%ticker%: No timezone found, symbol may be delisted')
['SEER3T.SA', 'CVCB3T.SA', 'ANIM3T.SA']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2020-01-01 00:00:00 -> 2023-12-10 00:00:00)')
['MRSA6BF.SA']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")


In [25]:
import utils
start = datetime(2020,1,1)
end = datetime(2023,12,10)

path_df = "s3://bbs-datalake/SourceZone/df_stocks_info.parquet"
df_stocks_info = pd.read_parquet(path_df)

stock_type = ['BDR', 'ON', 'PN', 'PNA', 'PNB', 'Subscrição', 'PNC', 'PND']
stock_sector = list(df_stocks_info["Setor"].unique())
stock_technical = ["Compra Forte", "Compra", "Venda", "Venda Forte", "Neutro"]

df_filter = df_stocks_info.query(
    "Tipo in @stock_type and Setor in @stock_sector and Mensal in @stock_technical"
)
stocks_codes = [i + ".SA" for i in df_filter.Códigos.unique()]
utils.collect_historico_stocks(stocks_codes, start, end)

2023-12-31 05:42:15.773 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-12-31 05:42:18.676 
  command:

    streamlit run c:\Users\bruno\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2023-12-31 05:42:18.678 No runtime found, using MemoryCacheStorageManager


[*********************100%%**********************]  981 of 981 completed


10 Failed downloads:
['1BHIA3.SA', '1WIZC3.SA', 'BPAN12.SA', '1ALOS3.SA', 'GOLL12.SA', '1HYPE3.SA']: Exception('%ticker%: No timezone found, symbol may be delisted')
['SEER3T.SA', 'CVCB3T.SA', 'ANIM3T.SA']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2020-01-01 00:00:00 -> 2023-12-10 00:00:00)')
['MRSA6BF.SA']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")


,AALL34.SA,AALR3.SA,AAPL34.SA,ABBV34.SA,ABCB4.SA,ABEV3.SA,ABTT34.SA,ACNB34.SA,ADBE34.SA,AGRO3.SA,ALPA3.SA,ALPA4.SA,ALUP3.SA,ALUP4.SA,AMAR3.SA,AMER3.SA,AMGN34.SA,AMZO34.SA,ANIM3.SA,ARMT34.SA,ARZZ3.SA,ASML34.SA,ATMP3.SA,ATOM3.SA,ATTB34.SA,AVGO34.SA,AXPB34.SA,AZEV3.SA,AZEV4.SA,AZUL4.SA,B3SA3.SA,BABA34.SA,BALM4.SA,BBAS3.SA,BBDC3.SA,BBDC4.SA,BBSE3.SA,BCSA34.SA,BEEF3.SA,BEES3.SA,BEES4.SA,BERK34.SA,BGIP4.SA,BHIA3.SA,BIDU34.SA,BIOM3.SA,BKNG34.SA,BLAK34.SA,BMEB4.SA,BMGB4.SA,BMIN4.SA,BMYB34.SA,BOAC34.SA,BOBR4.SA,BOEI34.SA,BONY34.SA,BOXP34.SA,BPAC3.SA,BPAC5.SA,BPAN4.SA,BRAP3.SA,BRAP4.SA,BRFS3.SA,BRIV3.SA,BRKM3.SA,BRKM5.SA,BRSR3.SA,BRSR6.SA,BSLI3.SA,BSLI4.SA,CAMB3.SA,CAML3.SA,CATP34.SA,CCRO3.SA,CEAB3.SA,CEBR3.SA,CEBR6.SA,CEDO4.SA,CGAS5.SA,CGRA4.SA,CHCM34.SA,CHVX34.SA,CIEL3.SA,CLSC3.SA,CLSC4.SA,CMCS34.SA,CMIG3.SA,CMIG4.SA,CNIC34.SA,COCA34.SA,COCE5.SA,COGN3.SA,COLG34.SA,COPH34.SA,COWC34.SA,CPFE3.SA,CPLE3.SA,CPLE6.SA,CRFB3.SA,CRIV4.SA,CRPG5.SA,CSAN3.SA,CSMG3.SA,CSNA3.SA,CSRN3.SA,CSUD3.SA,CSXC34.SA,CTGP34.SA,CTNM4.SA,CTSA3.SA,CTSA4.SA,CVCB3.SA,CVSH34.SA,CYRE3.SA,DASA3.SA,DEAI34.SA,DEEC34.SA,DEXP3.SA,DHER34.SA,DIRR3.SA,DISB34.SA,DOHL4.SA,DXCO3.SA,EAIN34.SA,EALT4.SA,ECOR3.SA,EGIE3.SA,ELCI34.SA,ELET3.SA,ELET6.SA,EMAE4.SA,EMBR3.SA,ENAT3.SA,ENEV3.SA,ENGI3.SA,ENGI4.SA,EQIX34.SA,EQPA3.SA,EQTL3.SA,ESTR4.SA,ETER3.SA,EUCA3.SA,EUCA4.SA,EVEN3.SA,EXXO34.SA,EZTC3.SA,FCXO34.SA,FDMO34.SA,FDXB34.SA,FESA4.SA,FHER3.SA,FLRY3.SA,FRAS3.SA,FRTA3.SA,FSLR34.SA,GEOO34.SA,GFSA3.SA,GGBR3.SA,GGBR4.SA,GILD34.SA,GMCO34.SA,GOAU3.SA,GOAU4.SA,GOGL34.SA,GOGL35.SA,GOLL4.SA,GPIV33.SA,GRND3.SA,GSGI34.SA,GUAR3.SA,HAGA3.SA,HAGA4.SA,HAPV3.SA,HBOR3.SA,HOME34.SA,HONB34.SA,HOND34.SA,HOOT4.SA,HPQB34.SA,HYPE3.SA,IBMB34.SA,IGTI3.SA,INEP3.SA,INEP4.SA,IRBR3.SA,ITLC34.SA,ITSA3.SA,ITSA4.SA,ITUB3.SA,ITUB4.SA,JBSS3.SA,JDCO34.SA,JFEN3.SA,JHSF3.SA,JNJB34.SA,JOPA3.SA,JPMC34.SA,KEPL3.SA,KHCB34.SA,KLBN3.SA,KLBN4.SA,LEVE3.SA,LIGT3.SA,LILY34.SA,LOGG3.SA,LOGN3.SA,LPSB3.SA,LREN3.SA,LUPA3.SA,LUXM4.SA,M1TA34.SA,MACY34.SA,MCDC34.SA,MCOR34.SA,MDIA3.SA,MDTC34.SA,MEAL3.SA,MELI34.SA,MERC4.SA,MGEL4.SA,MGLU3.SA,MILS3.SA,MMMC34.SA,MNDL3.SA,MNPR3.SA,MOAR3.SA,MOOO34.SA,MOSC34.SA,MOVI3.SA,MRCK34.SA,MRFG3.SA,MRVE3.SA,MSBR34.SA,MSCD34.SA,MSFT34.SA,MTSA4.SA,MULT3.SA,MUTC34.SA,MWET4.SA,MYPK3.SA,NEOE3.SA,NETE34.SA,NEXP3.SA,NEXT34.SA,NFLX34.SA,NIKE34.SA,NTCO3.SA,NUTR3.SA,NVDC34.SA,ODPV3.SA,OFSA3.SA,OIBR3.SA,OIBR4.SA,ORCL34.SA,OSXB3.SA,OXYP34.SA,PCAR3.SA,PDGR3.SA,PDTC3.SA,PEPB34.SA,PETR3.SA,PETR4.SA,PFIZ34.SA,PFRM3.SA,PGCO34.SA,PHMO34.SA,PINE4.SA,PMAM3.SA,PNVL3.SA,POMO3.SA,POMO4.SA,POSI3.SA,PRIO3.SA,PSSA3.SA,PTBL3.SA,PTNT3.SA,PTNT4.SA,PYPL34.SA,QCOM34.SA,QUAL3.SA,RADL3.SA,RAIL3.SA,RANI3.SA,RAPT3.SA,RAPT4.SA,RCSL3.SA,RCSL4.SA,RDNI3.SA,REDE3.SA,REGN34.SA,RENT3.SA,RIGG34.SA,RIOT34.SA,RNEW3.SA,RNEW4.SA,ROMI3.SA,RPMG3.SA,RSID3.SA,RSUL4.SA,RYTT34.SA,SANB3.SA,SANB4.SA,SAPR3.SA,SAPR4.SA,SBFG3.SA,SBSP3.SA,SBUB34.SA,SCAR3.SA,SCHW34.SA,SEER3.SA,SGPS3.SA,SHOW3.SA,SHUL4.SA,SIMN34.SA,SLBG34.SA,SLCE3.SA,SMTO3.SA,SNEC34.SA,SNSY5.SA,SPGI34.SA,SSFO34.SA,STBP3.SA,SUZB3.SA,SYNE3.SA,TAEE3.SA,TAEE4.SA,TASA3.SA,TASA4.SA,TCSA3.SA,TECN3.SA,TEKA4.SA,TELB3.SA,TELB4.SA,TEND3.SA,TEXA34.SA,TGMA3.SA,TGTB34.SA,TIMS3.SA,TMCO34.SA,TMOS34.SA,TOTS3.SA,TPIS3.SA,TRIS3.SA,TRPL3.SA,TRPL4.SA,TSLA34.SA,TSMC34.SA,TUPY3.SA,TXRX4.SA,U1BE34.SA,UCAS3.SA,UGPA3.SA,ULEV34.SA,UNHH34.SA,UNIP3.SA,UNIP6.SA,UPAC34.SA,USBC34.SA,USIM3.SA,USIM5.SA,VALE3.SA,VBBR3.SA,VERZ34.SA,VISA34.SA,VIVA3.SA,VIVR3.SA,VIVT3.SA,VLID3.SA,VLOE34.SA,VRTX34.SA,VSTE3.SA,VULC3.SA,WALM34.SA,WEGE3.SA,WFCO34.SA,WGBA34.SA,WHRL3.SA,WHRL4.SA,WIZC3.SA,WUNI34.SA,YDUQ3.SA,ZAMP3.SA
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-02,115.406067,18.843966,14.781657,19.488880

In [28]:
#colunas que contém nan
colunas_nan = list(historico_stocks.iloc[:,list(historico_stocks.isna().any())].columns)
if colunas_nan!=[]:
    print(f'Os ativos {colunas_nan} contém dados faltantes, e portante não devem ser utilizadas no portfólio')
    historico_stocks = historico_stocks.drop(columns=colunas_nan)
    stocks = list(historico_stocks.columns)
else:
    stocks = list(historico_stocks.columns)

Os ativos ['1ALOS3.SA', '1BHIA3.SA', '1HYPE3.SA', '1WIZC3.SA', 'A1AP34.SA', 'A1CR34.SA', 'A1DI34.SA', 'A1DM34.SA', 'A1EE34.SA', 'A1EG34.SA', 'A1EN34.SA', 'A1EP34.SA', 'A1ES34.SA', 'A1FL34.SA', 'A1GI34.SA', 'A1IV34.SA', 'A1KA34.SA', 'A1LB34.SA', 'A1LG34.SA', 'A1LK34.SA', 'A1LL34.SA', 'A1LN34.SA', 'A1MD34.SA', 'A1ME34.SA', 'A1MP34.SA', 'A1MT34.SA', 'A1NE34.SA', 'A1NS34.SA', 'A1ON34.SA', 'A1PA34.SA', 'A1PD34.SA', 'A1PH34.SA', 'A1RE34.SA', 'A1SN34.SA', 'A1SU34.SA', 'A1TH34.SA', 'A1TM34.SA', 'A1TT34.SA', 'A1UT34.SA', 'A1VB34.SA', 'A1WK34.SA', 'A1YX34.SA', 'A1ZN34.SA', 'A2FY34.SA', 'ABUD34.SA', 'ADPR34.SA', 'AERI3.SA', 'AESB3.SA', 'AFLT3.SA', 'AGXY3.SA', 'AHEB3.SA', 'AIGB34.SA', 'AIRB34.SA', 'ALLD3.SA', 'ALOS3.SA', 'ALPK3.SA', 'AMBP3.SA', 'ANIM3T.SA', 'APER3.SA', 'APTV34.SA', 'ARML3.SA', 'ARNC34.SA', 'ASAI3.SA', 'AURA33.SA', 'AURE3.SA', 'AVLL3.SA', 'B1AM34.SA', 'B1AX34.SA', 'B1BT34.SA', 'B1CS34.SA', 'B1DX34.SA', 'B1GN34.SA', 'B1IL34.SA', 'B1KR34.SA', 'B1LL34.SA', 'B1MR34.SA', 'B1NT34.SA', 'B

In [35]:
(historico_stocks.values>0).any()

True

In [92]:
def df_moving_avg(df, sma=25):
    return df.rolling(sma).mean().dropna()

In [93]:
def df_optimal_pso_points(df, num_points_to_choose = 30, num_particles = 15, num_iterations = 40):
    stocks_series_list = []
    for stock in list(df.columns):
        stock_series = df[stock]
        chosen_indices = PSO_optimal_points_stocks.pso(num_particles, num_iterations)
        sorted_v = sorted(range(len(chosen_indices)), key=lambda k: chosen_indices[k])
        sorted_indices = [chosen_indices[i] for i in sorted_v]
        chosen_dates = [stock_series.index[i] for i in sorted_indices]
        stock_series_opt_points = stock_series[chosen_dates]
        stocks_series_list.append(stock_series_opt_points)
        placeholder_df = pd.DataFrame(index=df.index)
        pd_new = pd.concat(stocks_series_list+[placeholder_df], axis=1).interpolate(limit_direction='both')
    return pd_new

In [94]:
def df_weighted(df, recent_weight=.8):
    new_df = df.copy()
    for stock in list(df.columns):
        new_df[stock] = np.linspace(1, recent_weight, len(df[stock]))[::-1]*df[stock]
    return new_df

In [187]:
historico_stocks_weighted = df_weighted(historico_stocks)
historico_stocks_optimal_pso_points = df_optimal_pso_points(historico_stocks)
historico_stocks_moving_avg = df_moving_avg(historico_stocks)

In [38]:
fig = go.Figure()
for stock in stocks:
    fig.add_trace(go.Scatter(x=historico_stocks.index, y=historico_stocks[stock],
                                     mode='lines',
                                     name=f'{stock}'))
fig.update_layout(title='Histórico de fechamento das ações ', 
                    xaxis_title='Período',
                    yaxis_title='Preço')
fig.write_html("Histórico.html")

<h3> Calcula o retorno e risco das ações </h3>

<p> O principal aspecto da teoria do portfólio é que o risco individual de um ativo é diferente de seu risco na carteira, tornando a diversificação capaz de minimizar o risco não-sistemático dos ativos em conjunto. Com a minimização, é possível escolher a proporção ideal de cada ativo no portfólio, otimizando a relação retorno/risco da carteira de títulos. A figura abaixo representa bem essa ideia: Para mais de 30 ativos, é possível mitigar praticamente todo o risco não-sistemático da carteira. O resto é risco de mercado, crédito, liquidez ou operacional. </p>

<img src="https://media-exp1.licdn.com/dms/image/C4D12AQHx1Wy9yxJRlw/article-inline_image-shrink_1000_1488/0/1559501707529?e=1629936000&v=beta&t=ZyiAkVi7HhjbCx_dU-q1Q6A2AITVAudFLkWk1Iuvyn0">

In [40]:
data = historico_stocks.values.transpose()
data = np.where(data < 0, 1e-07, data)
Returns = np.zeros((data.shape[1]-1,data.shape[0]))
for i in range(Returns.shape[1]):
    epsilon = 1e-12
    Returns[:,i] = np.log(data[i,1::]/data[i,0:-1])+1
Returns = Returns[:,:-1]

In [41]:
wE = 221 ; #Janela de estimativa
R = Returns[::-1,:] #(a:k:b) a = primeiro indice, k = step size, b = último
#R = R[0:wE+1,:]

O retorno esperado de um ativo é dado por 
$$E(R) = \frac{1}{n}\sum_{i=1}^{n}R_{t},$$
onde $n$ é o tamanho do período histórico considerado e $R_{t}$ o retorno em cada instante $t$ do histórico. O retorno da
carteira $\bar{R}_{c}$, é média dos retornos esperados de cada ativo.

In [42]:
ExpR = np.mean(R, axis=0).reshape(-1,1).T #retorno esperado
MeanR = numpy.matlib.repmat(ExpR,R.shape[0],1)

<h3> Minimiza o VaR e CVaR </h3>

<p> Aqui a ideia é entender o risco como o quanto você aceita perder. Muito simples não? Suponha que você vai fazer um investimento de $1000$ reais. Seu gerente lhe diz que na carteira $X$ você pode ter um retorno de $300$% no ano e que você pode perder no máximo $900$ reais com chance de $5$%. O VaR é a perda máxima esperada (não confundir com a perda máxima possível), os $900$ reais, e o alfa do VaR é a chance de você perder mais que isso (no exemplo $5$%). Minimizar o VaR significa escolher o melhor conjunto de ativos que, com um mesmo retorno, diminua essa perda máxima esperada. </P>

<p> Como já mencionei, o VaR falha na subaditividade. E pior, falha numa propriedade que a galera da otimização adora (eu também!), a convexidade. Felizmente, temos uma medida que, além de ser convexa, é coerente. O Valor em risco condicional (CVaR) examina as perdas que excedem o limite do Valor em Risco (VaR). No exemplo que demos da carteira $X$, isso significa analisar as perdas para 5%, 4%,... de chance e tirar uma média disso. O VaR e o CVaR estão intimamente relacionados e, ao minimizar o CVaR, também levará a uma redução do VaR da carteira. A figura abaixo expressa uma curva normal com as perdas esperadas do Var e CVaR e as probabilidades esperadas. </p>

<ol>
<li>O VaR tenta resumir em um único número ($\alpha$), a perda máxima esperada dentro de um certo prazo com um certo grau de confiança estatística: $$VaR_{1-\alpha}(X):=inf\left \{t\in \mathbb{R} : Pr(X\leqslant  t)\geqslant 1-\alpha\right \}, \alpha\in [0,1]$$ </li>
<li>O CVaR pode ser definido como a esperança condicional de perdas das carteiras superiores ao VaR: 
 $$CVaR_{1-\alpha}(X):=inf\left (t+\frac{1}{\alpha}E[X-t]_{+} \right ), \alpha\in (0,1]$$ $$CVaR_{1-\alpha}(X)=\frac{1}{\alpha}\int_{0}^{\alpha}VaR_{1-t}(X)dt,$$ onde $[c]_{+}=max(0,c).$</li>

<img src="https://media-exp1.licdn.com/dms/image/C4D12AQFg17zgxgCBUQ/article-inline_image-shrink_1000_1488/0/1559505124297?e=1629936000&v=beta&t=AKHOCeCHUtinkk2eEB_lIlWH-6G1Stb4TC7BQxsvtWk">

A partir de uma série da manipulações matemáticas, Rockafellar e Uryasev (2000) reescreve o cálculo do CVaR em termos de uma função $F_{\beta}$, dada por
$$F_{\beta}(x,\alpha)=\alpha+(1-\beta)^{-1}\int_{y\in \mathbb{R}^{m}}[f(x,y)-\alpha]p(y)dy,$$ onde $p(y)$ é a função densidade de probabilidade de variáveis de mercado; $\beta$ é o nível de probabilidade a ser escolhido; e $f(x,y)$ é uma função de perda associada à carteira $x$ e a variáveis de mercado $y$.

Para o caso de valores discretos, a equação acima pode ser reescrita como:
$$F_{\beta}(x,\alpha)=\alpha+\frac{1}{n(1-\beta)}\sum_{k=1}^{n}[f(x,y)-\alpha]_{+}$$

Dessa maneira, Rockafellar e Uryasev (2000) usa a função $F_{\beta}$ linear para definir a forma para a otimização de uma carteira de ações utilizando o CVaR como medida de risco:
$$\begin{align*}
 Min\;\; \alpha+\frac{1}{n(1-\beta)}\sum_{k=1}^{n}[-w_{i}R_{i}-\alpha]_{+}
 \\ 
 S.a\;\; \sum_{k=1}^{n}w_{i}=1 \\ 
 0\leqslant w_{i}\leq 1 ,
\end{align*}$$
one $n$ é o tamanho da amostra e $w_{i}$ a proporção de cada ativo na carteira.

In [44]:
N = ExpR.shape[1] #número de ativos

nS = Returns.shape[0] #número de cenários
V0 = 1 #Initial wealth
a = 0.95 #Confidence level
h = 21 #Número de day trades por mês
meanR = np.linspace(np.min(ExpR),np.max(ExpR),50)
#Allocation for linprog output
linMap = np.zeros((nS+N+1,len(meanR)))
f = np.vstack((np.zeros((N,1)),1,1/((1-a)*nS)*np.ones((nS,1))))
#Coeficientes da restrição de desigualdade
w = np.ones((nS,N)) - Returns
v = -np.ones((nS,1))
y = -np.eye(nS)
A = np.hstack([w,v,y])
b = np.zeros((nS,1))
#Restrições do coeficiente de igualdade
Aeq = np.vstack([np.hstack([ExpR,np.array([0]).reshape(-1,1),np.zeros((1,nS))]),np.hstack([np.ones((1,N)),np.array([0]).reshape(-1,1),np.zeros((1,nS))])])
#Restrições de fronteira
lb = np.zeros((1,nS+N+1))
ub = np.full((1,nS+N+1), np.inf)
bounds = np.vstack([lb,ub]).transpose()
    
for i in tqdm(range(len(meanR))):
    beq = np.vstack([meanR[i]*V0,V0])
    linMap[:,i] = linprog(f,A,b,Aeq,beq,bounds, method='highs-ds')['x']          
VaR = linMap[N,:]
CVaR = sum(linMap[N+1::,:])/((1-a)*nS) + linMap[N,:]
w = linMap[0:ExpR.shape[1],np.argmin(CVaR)::]
meanR = 100*(meanR-1)*h
VaR = 100*VaR*np.sqrt(h)
CVaR = 100*CVaR*np.sqrt(h)



  0%|          | 0/50 [00:00<?, ?it/s]

In [56]:
len(VaR)

50

Os gráficos abaixo expressam o retorno para cada valor em risco considerado. Devido ao fato da otimização do VaR não ser convexa, a fronteira eficiente da otimização com o VaR tem um comportamento mais "caótico" comparada à otimização com o CVaR, que tem um comportamento mais suave.

In [71]:
fig_1 = go.Figure(data=go.Scatter(x=VaR[meanR>0], y=meanR[meanR>0]))
fig_1.update_layout(title='Value at Risk - Fronteira Eficiente', 
    xaxis_title='Value at Risk (%)',
    yaxis_title='Retorno Esperado (%)')
fig_1.show()

In [72]:
fig_2 = go.Figure(data=go.Scatter(x=CVaR[meanR>0], y=meanR[meanR>0]))
fig_2.update_layout(title='Conditional Value at Risk - Fronteira Eficiente', 
                    xaxis_title='Conditional Value at Risk (%)',
                    yaxis_title='Retorno Esperado (%)')
fig_2.show()

O gráfico de área abaixo expressa a proporção de ativos na carteira para cada valor em risco considerado.

In [73]:
x=CVaR[np.argmin(CVaR)::]
y = 100*w

fig_2 = go.Figure()
for i in range(len(y)):
    fig_2.add_trace(go.Scatter(
                    x=x, y=y[i],
                    mode='lines',
                    line=dict(width=0.5),
                    name=stocks[i],
                    stackgroup='one'))
fig_2.update_layout(title='Distribuição de Ações na carteira - CVAR', 
                    xaxis_title='Conditional Value at Risk (%)',
                    yaxis_title='Distribuição (%)')
fig_2.write_html("proportional.html")

In [74]:
def proportion_CVAR(CVAR_value, CVaR_portfolio, y, stocks):
    i = np.argmin(abs(CVaR_portfolio-CVAR_value))
    CVAR = {}
    for j, stock in zip(range(len(stocks)-1), stocks):
        CVAR[stock] = round(y[j][i],2)
    CVAR = pd.Series(CVAR)
    CVAR = CVAR[CVAR!=0].sort_values()
    fig = px.bar(CVAR, y=CVAR.index,x=CVAR, orientation='h')
    fig.update_layout(
    title='Proporção de ativos na carteira',
    xaxis_title='Proporção',
    yaxis_title= 'Ações')
    fig.write_html("stocks.html")
    return CVAR, fig

O resultado final é, para um dado valor em risco, as ações que devo investir e em qual proporção, para o meu portfólio ótimo.

In [75]:
proportion_CVAR(20, x, y, stocks)[1]

<h3> Backtest </h3>

O objetivo do backtest é analisar quais seriam os resultados de um carteira ótima criada num período anterior ao atual, verificando a evolução da carteira desde o instante em que ela foi criada, até o instante atual.

In [99]:
def drop_columns_with_nan(df):
    # Identify columns with NaN values
    columns_nan = list(df.iloc[:, list(df.isna().any())].columns)

    # Check if there are columns with NaN values
    if columns_nan != []:
        print(
            f"The assets {columns_nan} contain missing data and should not be used in the portfolio."
        )
        
        # Drop columns with NaN values
        df_new = df.drop(columns=columns_nan)
    else:
        # If no NaN values are found, return the original dataframe
        return df
    
    # Return the dataframe without columns with NaN values
    return df_new




def collect_historico_stocks(stocks_codes, data_inicio, data_fim):
    historico_stocks = web.DataReader(stocks_codes, data_inicio, data_fim)["Adj Close"]
    historico_stocks = drop_columns_with_nan(historico_stocks)
    return historico_stocks


class Portfolio_optimization:
    def __init__(self, historico_stocks):
        self.historico_stocks = historico_stocks
        self.stocks = list(historico_stocks.columns)

    # Function to plot historical stock prices
    def plot_historic(self):
        fig = go.Figure()

        # Iterate through each stock in the portfolio
        for stock in self.stocks:
            # Add a trace for each stock, representing historical closing prices
            fig.add_trace(
                go.Scatter(
                    x=self.historico_stocks.index,
                    y=self.historico_stocks[stock],
                    mode="lines",
                    name=f"{stock}",
                )
            )

        # Update layout with title and axis labels
        fig.update_layout(
            title="Historical Closing Prices of Stocks",
            xaxis_title="Time Period",
            yaxis_title="Price",
        )

        # Display the Plotly chart using Streamlit
        fig.show()

    def returns(self):
        '''This function calculates logarithmic returns for each stock based on historical stock data, 
        considers a specified window length (wE), and returns a dictionary containing the calculated returns, 
        expected returns, and mean returns.'''
        # Transpose historical stock data for easier calculation
        data = self.historico_stocks.values.transpose()
        data = np.where(data < 0, 1e-07, data)
        # Initialize an array to store logarithmic returns
        Returns = np.zeros((data.shape[1] - 1, data.shape[0]))

        # Calculate logarithmic returns for each stock
        for i in range(Returns.shape[1]):
            Returns[:, i] = np.log(data[i, 1::] / data[i, 0:-1]) + 1

        # Keep only the necessary data based on the specified window length (wE)
        R = Returns[::-1, :]  # Reverse the array
        #R = R[0 : wE + 1, :]  # Slice to include the specified window length

        # Calculate expected returns and mean return
        ExpR = np.mean(R, axis=0).reshape(-1, 1).T  # Expected return
        MeanR = numpy.matlib.repmat(ExpR, R.shape[0], 1)  # Repeat expected return to match the shape of Returns

        # Return a dictionary containing Returns, Expected Returns, and Mean Return
        return {"Returns": Returns, "Expected Returns": ExpR, "Mean Return": MeanR}
    

    def optimize(self, Returns, ExpR, a=0.95, h=21, metric="CVaR"):
        '''This function optimizes a portfolio based on a specified risk metric (VaR or CVaR) 
        using linear programming. It calculates the mean return values and the corresponding 
        risk measure values for a range of possible returns. The optimized portfolio weights and 
        the calculated risk measure are then returned as a dictionary.'''
        N = ExpR.shape[1]  # Number of assets
        nS = Returns.shape[0]  # Number of scenarios
        V0 = 1  # Initial wealth
        meanR = np.linspace(np.min(ExpR), np.max(ExpR), 50)  # Mean return values for optimization
        linMap = np.zeros((nS + N + 1, len(meanR)))  # Allocation for linprog output

        # Objective function coefficients
        f = np.vstack((np.zeros((N, 1)), 1, 1 / ((1 - a) * nS) * np.ones((nS, 1))))

        # Coefficients for inequality constraints
        w = np.ones((nS, N)) - Returns
        v = -np.ones((nS, 1))
        y = -np.eye(nS)
        A = np.hstack([w, v, y])
        b = np.zeros((nS, 1))

        # Coefficients for equality constraints
        Aeq = np.vstack(
            [
                np.hstack([ExpR, np.array([0]).reshape(-1, 1), np.zeros((1, nS))]),
                np.hstack([np.ones((1, N)), np.array([0]).reshape(-1, 1), np.zeros((1, nS))]),
            ]
        )

        # Constraints for boundary values
        lb = np.zeros((1, nS + N + 1))
        ub = np.full((1, nS + N + 1), np.inf)
        bounds = np.vstack([lb, ub]).transpose()

        # Loop through mean return values and solve linear programming problem
        for i in tqdm(range(len(meanR))):
            beq = np.vstack([meanR[i] * V0, V0])
            linMap[:, i] = linprog(f, A, b, Aeq, beq, bounds, method="highs-ds")["x"]

        # Adjust mean return values for display
        meanR = 100 * (meanR - 1) * h

        # Calculate risk measure based on the chosen metric (VaR or CVaR)
        if metric == "VaR":
            v = linMap[N, :]
            w = linMap[0 : ExpR.shape[1], np.argmin(v) : :]
            v = 100 * v * np.sqrt(h)
        elif metric == "CVaR":
            v = sum(linMap[N + 1 :, :]) / ((1 - a) * nS) + linMap[N, :]
            w = linMap[0 : ExpR.shape[1], np.argmin(v) : :]
            v = 100 * v * np.sqrt(h)

        # Return the results as a dictionary
        return {"meanR": meanR, "risk_measure": v, 'w':w}

    # Function to plot efficient frontiers based on risk metric (VaR or CVaR)
    def plot_efficient_frontiers(self, values, meanR, metric="CVaR"):
        # Create a Plotly figure with a scatter plot
        fig_1 = go.Figure(data=go.Scatter(x=values[meanR > 0], y=meanR[meanR > 0]))

        # Update layout with title and axis labels
        fig_1.update_layout(
            title=f"{metric} - Efficient Frontier",
            xaxis_title=f"{metric} (%)",
            yaxis_title="Expected Return (%)",
        )

        # Markdown explanation about the efficient frontiers
        print(
            """The charts below represent the return for each value of the considered risk. 
            Due to the non-convex nature of VaR optimization, the efficient frontier of VaR optimization 
            exhibits a more "chaotic" behavior compared to CVaR optimization, which has a smoother behavior."""
        )

        # Display the Plotly chart using Streamlit
        fig_1.show()

    # Function to plot the distribution of stocks in the portfolio
    def plot_stocks_distribution(self, values, w, metric="CVaR", plot=True):
        # Extract data for x-axis (risk values)
        x = values[np.argmin(values) : :]

        # Extract portfolio allocation data for y-axis
        y = 100 * w

        # Create a Plotly figure
        fig_3 = go.Figure()

        # Iterate through each stock and add a trace for the stock's distribution
        for i in range(len(y)):
            fig_3.add_trace(
                go.Scatter(
                    x=x,
                    y=y[i],
                    mode="lines",
                    line=dict(width=0.5),
                    name=self.stocks[i],
                    stackgroup="one",
                )
            )

        # Update layout with title and axis labels
        fig_3.update_layout(
            title=f"Stocks Distribution in Portfolio - {metric}",
            xaxis_title=f"{metric} (%)",
            yaxis_title="Distribution (%)",
        )

        # Display a markdown explanation if the plot parameter is True
        if plot:
            print('''The area chart below represents the proportion of assets in 
                        the portfolio for each considered risk value.''')
            fig_3.show()

        # Return the x and y data
        return x, y

    # Function to visualize the proportion of assets in the portfolio for a given risk value
    def proportion_risk(self, risk_value, x, y, plot=True):
        # Find the index corresponding to the given risk value
        i = np.argmin(abs(x - risk_value))

        # Create a dictionary to store the proportion of each stock at the specified risk value
        risk = {}
        for j, stock in enumerate(self.stocks):
            risk[stock] = round(y[j][i], 2)

        # Convert the dictionary to a pandas Series for easier manipulation
        risk = pd.Series(risk)

        # Create a bar chart using Plotly Express
        risk_2 = risk[risk != 0].sort_values()
        fig_4 = px.bar(risk_2, y=risk_2.index, x=risk_2, orientation="h")

        # Update layout with title and axis labels
        fig_4.update_layout(
            title="Proportion of Assets in Portfolio",
            xaxis_title="Proportion",
            yaxis_title="Stocks",
        )

        # Display a markdown explanation if the plot parameter is True
        if plot:
            print('''The final result is, for a given risk value, 
                        the stocks I should invest in and in what proportion, for my optimal portfolio.''')
            fig_4.show()

        # Return the proportion of assets for each stock at the specified risk value
        return risk



def backtest(
    valor_investido,
    stocks_codes,
    risk_values,
    start_date_backtest,
    end_date_backtest,
    current_date,
):
    '''This function performs a backtest on a portfolio optimization strategy, 
    calculating the value of the portfolio over time for different levels of risk. 
    The results are visualized using a Plotly figure.'''
    # Collect historical stock data for the backtest period
    historico_stocks = collect_historico_stocks(
        stocks_codes, start_date_backtest, current_date
    )

    # Split the data into training and testing sets
    train = historico_stocks[:end_date_backtest]
    test = historico_stocks[end_date_backtest:]

    # Initialize a Portfolio_optimization object for training data
    po = Portfolio_optimization(train)

    # Calculate returns and optimize portfolio for training data
    r_dict = po.returns()
    Returns = r_dict["Returns"]
    ExpR = r_dict["Expected Returns"]
    opt_dict = po.optimize(Returns, ExpR, a=0.95, h=21, metric="CVaR")
    risk_measure = opt_dict["risk_measure"]
    w = opt_dict["w"]

    # Plot stocks distribution for training data
    x, y = po.plot_stocks_distribution(risk_measure, w, metric="CVaR")

    # Generate portfolios for different risk values
    portfolios = [
        po.proportion_risk(risk_value, x, y, False) for risk_value in risk_values
    ]

    fig = go.Figure()

    # Iterate through each portfolio and perform the backtest
    for portfolio, risco in zip(portfolios, risk_values):
        inv_inicio = test.iloc[0]
        inv_fim = test.iloc[-1]
        a = portfolio / inv_inicio * inv_fim
        valor_atual = sum(a) / 100 * valor_investido

        # Display information about the backtest results
        print(f"Initial Investment: {valor_investido}")
        print(
            f"For risk = {risco}, the current value of the investment is {round(valor_atual, 2)}"
        )

        # Calculate the portfolio value at each time point in the testing set
        valor = {}
        for i in test.index:
            inv_fim = test.loc[i]
            v = sum(portfolio / inv_inicio * inv_fim) / 100 * valor_investido
            valor[i] = v

        # Create a trace for each portfolio in the Plotly figure
        fig.add_trace(
            go.Scatter(x=list(valor.keys()), y=list(valor.values()), mode="lines", name=f"Risk = {risco}")
        )

    # Update layout with title and axis labels for the Plotly figure
    fig.update_layout(
        title="Portfolio Value Evolution",
        xaxis_title="Time Period",
        yaxis_title="Portfolio Value",
    )

    # Display the Plotly figure using Streamlit
    fig.show()


In [100]:
data_inicio = datetime(2020, 1, 1)
data_fim = datetime.now()
stock_type = ["ON"]
stock_sector = list(df_stocks_info["Setor"].unique())
stock_tecnichal = ["Compra Forte"]
df_filter = df_stocks_info.query(
    "Tipo in @stock_type and Setor in @stock_sector and Mensal in @stock_tecnichal"
)
stocks_codes = [i + ".SA" for i in df_filter.Códigos.unique()]
days_before = 60
end_date_backtest = datetime.now() - timedelta(days=days_before)
invested_value = 5000
risk_values = [20,30,40,50,60]

backtest(
        invested_value,
        stocks_codes,
        risk_values,
        data_inicio,
        end_date_backtest,
        data_fim,
    )

[*********************100%%**********************]  149 of 149 completed


2 Failed downloads:
['1WIZC3.SA', '1ALOS3.SA']: Exception('%ticker%: No timezone found, symbol may be delisted')



The assets ['1ALOS3.SA', '1WIZC3.SA', 'AESB3.SA', 'ALOS3.SA', 'ALPK3.SA', 'APER3.SA', 'BAZA3.SA', 'BMEB3.SA', 'BMKS3.SA', 'BNBR3.SA', 'BRGE3.SA', 'BRIT3.SA', 'CALI3.SA', 'CEDO3.SA', 'CEEB3.SA', 'CMIN3.SA', 'CRIV3.SA', 'CSED3.SA', 'CURY3.SA', 'CXSE3.SA', 'DESK3.SA', 'DMVF3.SA', 'EALT3.SA', 'ELMD3.SA', 'ENJU3.SA', 'EQMA3B.SA', 'FESA3.SA', 'FRIO3.SA', 'GEPA3.SA', 'GGPS3.SA', 'GMAT3.SA', 'GPAR3.SA', 'HBRE3.SA', 'HBSA3.SA', 'JSLG3.SA', 'LAVV3.SA', 'LIPR3.SA', 'LJQQ3.SA', 'MDNE3.SA', 'MELK3.SA', 'NINJ3.SA', 'ONCO3.SA', 'OPCT3.SA', 'ORVR3.SA', 'PINE3.SA', 'PLPL3.SA', 'PORT3.SA', 'PRNR3.SA', 'RPAD3.SA', 'SIMH3.SA', 'SMFT3.SA', 'SOJA3.SA'] contain missing data and should not be used in the portfolio.


  0%|          | 0/50 [00:00<?, ?it/s]

The area chart below represents the proportion of assets in 
                        the portfolio for each considered risk value.


Initial Investment: 5000
For risk = 20, the current value of the investment is 5778.34
Initial Investment: 5000
For risk = 30, the current value of the investment is 5879.49
Initial Investment: 5000
For risk = 40, the current value of the investment is 5879.49
Initial Investment: 5000
For risk = 50, the current value of the investment is 5879.49
Initial Investment: 5000
For risk = 60, the current value of the investment is 5879.49


In [5]:
import numpy as np
np.linspace(1, .2, 100)[::-1]

array([0.2       , 0.20808081, 0.21616162, 0.22424242, 0.23232323,
       0.24040404, 0.24848485, 0.25656566, 0.26464646, 0.27272727,
       0.28080808, 0.28888889, 0.2969697 , 0.30505051, 0.31313131,
       0.32121212, 0.32929293, 0.33737374, 0.34545455, 0.35353535,
       0.36161616, 0.36969697, 0.37777778, 0.38585859, 0.39393939,
       0.4020202 , 0.41010101, 0.41818182, 0.42626263, 0.43434343,
       0.44242424, 0.45050505, 0.45858586, 0.46666667, 0.47474747,
       0.48282828, 0.49090909, 0.4989899 , 0.50707071, 0.51515152,
       0.52323232, 0.53131313, 0.53939394, 0.54747475, 0.55555556,
       0.56363636, 0.57171717, 0.57979798, 0.58787879, 0.5959596 ,
       0.6040404 , 0.61212121, 0.62020202, 0.62828283, 0.63636364,
       0.64444444, 0.65252525, 0.66060606, 0.66868687, 0.67676768,
       0.68484848, 0.69292929, 0.7010101 , 0.70909091, 0.71717172,
       0.72525253, 0.73333333, 0.74141414, 0.74949495, 0.75757576,
       0.76565657, 0.77373737, 0.78181818, 0.78989899, 0.79797